# TVM 拓展：Python 调用 C++

下面逐步揭开 TVM 中 C++/C 与 Python 交互的机制。

在 C++ 中定义加法算子：

````{tab-set-code}
```{literalinclude} cpp/sym_add/src/tvm_ext.cc
:language: c++
```

```{literalinclude} cpp/sym_add/Makefile
:language: Makefile
```
````

编译：

In [1]:
%%bash
cd cpp/sym_add
make clean
make

rm -rf outputs/*
g++ -std=c++17 -O2 -fPIC -I/media/pc/data/lxw/ai/tvm/include -I/media/pc/data/lxw/ai/tvm/3rdparty/dmlc-core/include -I/media/pc/data/lxw/ai/tvm/3rdparty/dlpack/include -Iinclude -DDMLC_USE_LOGGING_LIBRARY=\<tvm/runtime/logging.h\> -shared -o outputs/libs/libtvm_ext.so src/tvm_ext.cc -ldl -pthread -L/media/pc/data/lxw/ai/tvm/build


## Python 端加载 C++ 端动态库

可以使用 {mod}`ctypes` 加载动态库：

In [2]:
import ctypes

# 作为全局加载，使全局外部符号对其他 dll 可见。
_LIB = ctypes.CDLL("cpp/sym_add/outputs/libs/libtvm_ext.so", ctypes.RTLD_GLOBAL)

OSError: cpp/sym_add/outputs/libs/libtvm_ext.so: undefined symbol: _ZNK3tvm7runtime6Object11DerivedFromEj

加载失败，是由于 `libtvm_ext.so` 是在 `libtvm.so` 基础上拓展的，故而需要先提前加载 `libtvm.so`，或者直接 `import tvm`：

In [3]:
import set_env
import tvm
import ctypes

# 作为全局加载，使全局外部符号对其他 dll 可见。
_LIB = ctypes.CDLL("cpp/sym_add/outputs/libs/libtvm_ext.so", ctypes.RTLD_GLOBAL)

加载动态库，也可以直接使用 {func}`~tvm_book.tvm_ext.libinfo._load_lib`：

In [4]:
import set_env
from tvm_book.tvm_ext.libinfo import _load_lib

_LIB_EXT, _LIB_EXT_NAME = _load_lib(name="libtvm_ext.so", search_path=["cpp/sym_add/outputs/libs"])

回调 C++ 函数：

In [5]:
import tvm
sym_add = tvm.get_global_func("tvm_ext.sym_add")

测试：

In [6]:
from tvm import te
a = te.var("x")
b = te.var("y")
c = sym_add(a, b)
assert c.a == a and c.b == b
print(c)

x + y


这些调用细节可以借助 FFI 机制进行隐藏。

## 使用 {func}`tvm._ffi._init_api` 管理 TVM 插件

In [7]:
import set_env
from tvm_book.tvm_ext.libinfo import _load_lib

_LIB_EXT, _LIB_EXT_NAME = _load_lib(name="libtvm_ext.so", search_path=["cpp/sym_add/outputs/libs"])

In [8]:
import tvm

tvm._ffi._init_api("tvm_ext", __name__)

下面便可以直接使用 `tvm_ext` 下的函数了：

In [9]:
sym_add

```{tip}
C++ 端调用 Python 程序的示例请移步 [C++ 部署](../../tutorials/deploy/cpp)。
```

## 其他 C++ 打包函数的例子

### 加法偏函数

```{literalinclude} cpp/bind_add/src/tvm_ext.cc
:language: c++
```

编译：

In [10]:
%%bash
cd cpp/bind_add
make clean
make

rm -rf outputs/*
g++ -std=c++17 -O2 -fPIC -I/media/pc/data/lxw/ai/tvm/include -I/media/pc/data/lxw/ai/tvm/3rdparty/dmlc-core/include -I/media/pc/data/lxw/ai/tvm/3rdparty/dlpack/include -Iinclude -DDMLC_USE_LOGGING_LIBRARY=\<tvm/runtime/logging.h\> -shared -o outputs/libs/libtvm_ext.so src/tvm_ext.cc -ldl -pthread -L/media/pc/data/lxw/ai/tvm/build


In [11]:
import set_env
from tvm_book.tvm_ext.libinfo import _load_lib
import tvm
_LIB_EXT, _LIB_EXT_NAME = _load_lib(name="libtvm_ext.so", search_path=["cpp/bind_add/outputs/libs"])
tvm._ffi._init_api("tvm_ext", __name__)

In [12]:
bind_add

In [13]:
def add(a, b):
    return a + b

f = bind_add(add, 7)
assert f(2) == 9

### C++ 外部设备的例子

```{literalinclude} cpp/device_api/src/tvm_ext.cc
:language: c++
```
编译：

In [14]:
%%bash
cd cpp/device_api
make clean
make

rm -rf outputs/*
g++ -std=c++17 -O2 -fPIC -I/media/pc/data/lxw/ai/tvm/include -I/media/pc/data/lxw/ai/tvm/3rdparty/dmlc-core/include -I/media/pc/data/lxw/ai/tvm/3rdparty/dlpack/include -Iinclude -DDMLC_USE_LOGGING_LIBRARY=\<tvm/runtime/logging.h\> -shared -o outputs/libs/libtvm_ext.so src/tvm_ext.cc -ldl -pthread -L/media/pc/data/lxw/ai/tvm/build


In [15]:
import set_env
import tvm
from tvm_book.tvm_ext.libinfo import _load_lib

_LIB_EXT, _LIB_EXT_NAME = _load_lib(name="libtvm_ext.so", search_path=["cpp/device_api/outputs/libs"])
tvm._ffi._init_api("tvm_ext", __name__)

In [16]:
import numpy as np
from tvm import te
n = 10
A = te.placeholder((n,), name="A")
B = te.compute((n,), lambda *i: A(*i) + 1.0, name="B")
s = te.create_schedule(B.op)

def check_llvm():
    f = tvm.build(s, [A, B], tvm.target.Target("ext_dev", "llvm"))
    dev = tvm.ext_dev(0)
    # launch the kernel.
    a = tvm.nd.array(np.random.uniform(size=n).astype(A.dtype), dev)
    b = tvm.nd.array(np.zeros(n, dtype=B.dtype), dev)
    f(a, b)
    np.testing.assert_allclose(b.numpy(), a.numpy() + 1)

check_llvm()

## 回调 C++ 端外部函数

```{literalinclude} cpp/extern_func/src/tvm_ext.cc
:language: c++
```
编译：

In [17]:
%%bash
cd cpp/extern_func
make clean
make

rm -rf outputs/*
g++ -std=c++17 -O2 -fPIC -I/media/pc/data/lxw/ai/tvm/include -I/media/pc/data/lxw/ai/tvm/3rdparty/dmlc-core/include -I/media/pc/data/lxw/ai/tvm/3rdparty/dlpack/include -Iinclude -DDMLC_USE_LOGGING_LIBRARY=\<tvm/runtime/logging.h\> -shared -o outputs/libs/libtvm_ext.so src/tvm_ext.cc -ldl -pthread -L/media/pc/data/lxw/ai/tvm/build


In [18]:
import set_env
import tvm
from tvm_book.tvm_ext.libinfo import _load_lib

_LIB_EXT, _LIB_EXT_NAME = _load_lib(name="libtvm_ext.so", search_path=["cpp/extern_func/outputs/libs"])
tvm._ffi._init_api("tvm_ext", __name__)

In [19]:
import numpy as np
from tvm import te
n = 10
A = te.placeholder((n,), name="A")
B = te.compute(
    (n,), lambda *i: tvm.tir.call_extern("float32", "TVMTestAddOne", A(*i)), name="B"
)
s = te.create_schedule(B.op)

def check_llvm():
    f = tvm.build(s, [A, B], "llvm")
    dev = tvm.cpu(0)
    # launch the kernel.
    a = tvm.nd.array(np.random.uniform(size=n).astype(A.dtype), dev)
    b = tvm.nd.array(np.zeros(n, dtype=B.dtype), dev)
    f(a, b)
    np.testing.assert_allclose(b.numpy(), a.numpy() + 1)

check_llvm()

## 提取外部 C++ 函数

```{literalinclude} cpp/mini_runtime/src/tvm_ext.cc
:language: c++
```
编译：

In [1]:
%%bash
cd cpp/mini_runtime
make clean
make

rm -rf outputs/*
g++ -std=c++17 -O2 -fPIC -I/media/pc/data/lxw/ai/tvm/include -I/media/pc/data/lxw/ai/tvm/3rdparty/dmlc-core/include -I/media/pc/data/lxw/ai/tvm/3rdparty/dlpack/include -Iinclude -DDMLC_USE_LOGGING_LIBRARY=\<tvm/runtime/logging.h\> -shared -o outputs/libs/libtvm_ext.so src/tvm_ext.cc -ldl -pthread -L/media/pc/data/lxw/ai/tvm/build


In [3]:
import set_env
import tvm
from tvm_book.tvm_ext.libinfo import _load_lib

_LIB, _LIB_NAME = _load_lib(name="libtvm_ext.so", search_path=["cpp/mini_runtime/outputs/libs"])
tvm._ffi._init_api("tvm_ext", __name__)

In [4]:
fdict = tvm._ffi.registry.extract_ext_funcs(_LIB.TVMExtDeclare)
assert fdict["mul"](3, 4) == 12

## 汇总 TVM 插件测试 demo

将上述插件集中到：

```
extension/src
    testing/
        _make.cc
        bind_add.cc
        device_api.cc
        extern_func.cc
        mini_runtime.cc
        sym_add.cc
    tvm_ext.cc
```
编译：

In [1]:
%%bash
make clean
make

rm -rf outputs/*
g++ -std=c++17 -O2 -fPIC -I/media/pc/data/lxw/ai/tvm/include -I/media/pc/data/lxw/ai/tvm/3rdparty/dmlc-core/include -I/media/pc/data/lxw/ai/tvm/3rdparty/dlpack/include -Iinclude -DDMLC_USE_LOGGING_LIBRARY=\<tvm/runtime/logging.h\> -shared -o outputs/libs/libtvm_ext.so src/tvm_ext.cc -ldl -pthread -L/media/pc/data/lxw/ai/tvm/build
g++ -std=c++17 -O2 -fPIC -I/media/pc/data/lxw/ai/tvm/include -I/media/pc/data/lxw/ai/tvm/3rdparty/dmlc-core/include -I/media/pc/data/lxw/ai/tvm/3rdparty/dlpack/include -Iinclude -DDMLC_USE_LOGGING_LIBRARY=\<tvm/runtime/logging.h\> -shared -o outputs/libs/libtvm_plugin_module.so src/plugin_module.cc -ldl -pthread -L/media/pc/data/lxw/ai/tvm/build


代码见 ：{mod}`tvm_book.tvm_ext.testing`。

In [1]:
import set_env
from tvm_book.tvm_ext.libinfo import _load_lib
_LIB, _LIB_NAME = _load_lib(name="libtvm_ext.so", search_path=["outputs/libs"])
from tvm_book.tvm_ext.testing import demo

比如：{func}`~tvm_book.tvm_ext.testing.demo.sym_add`：

In [2]:
from tvm import te
a = te.var("x")
b = te.var("y")
c = demo.sym_add(a, b)
assert c.a == a and c.b == b
print(c)

x + y
